# Data curation

### Import some necessary functions

In [3]:
import os
def return_sys_path():
    path = '.'
    for _ in range(5):
        if 'Pre_Processing' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path
def return_data_path():
    path = '.'
    for _ in range(5):
        if 'Data' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path

In [4]:
import pydicom, sys
sys.path.append(return_sys_path())
import numpy as np
import SimpleITK as sitk
from Pre_Processing.Distribute_Patients import Separate_files
from Pre_Processing.Dicom_RT_and_Images_to_Mask.Image_Array_And_Mask_From_Dicom_RT import Dicom_to_Imagestack
from Pre_Processing.Dicom_RT_and_Images_to_Mask.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image
from Pre_Processing.Make_Single_Images.Make_Single_Images_Class import run_main

## Finding the Data

### Find where we put our data

In [5]:
data_path = os.path.join(return_data_path(),'Data','Whole_Patients')
print('We have ' + str(len(os.listdir(data_path))) + ' patients!')

We have 20 patients!


## Ensuring contour fidelity...

### Note that we've set 'get_images_mask' to False, this means we won't be getting any of the image data, just looking at the dicom RT files

In [6]:
Dicom_Reader = Dicom_to_Imagestack(get_images_mask=False)

In [7]:
Dicom_Reader.down_folder(data_path)

### What ROI names do we have?

#### This will tell us all the unique roi names, hence all_rois

In [8]:
for roi in Dicom_Reader.all_rois:
    print(roi)

bma_liver
Liver_BMA_Program_4
tried_liver
best_liver
Liver


## Make contour associations

#### We have quite a few contour names here.. now, we can either change the ROI names in the RT files, or make an associations file

#### The associations file associates a contour name with another one {'Current contour':'Desired name'}

In [9]:
associations = {'Liver_BMA_Program_4':'Liver',
                'bma_liver':'Liver',
                'best_liver':'Liver',
                'tried_liver':'Liver'}

### Tell the Dicom_Reader that we want to set the associations, get the images and mask for contour 'Liver'

In [10]:
Dicom_Reader.set_associations(associations)
Dicom_Reader.set_get_images_and_mask(True)
Dicom_Reader.set_contour_names(['liver'])

### Re-write RTs
#### This is commented out, because if I run it, then the example above won't show any different contour names

In [ ]:
# Dicom_Reader.associations = associations
# for RT in Dicom_Reader.all_RTs:
#     Dicom_Reader.rewrite_RT(RT)

## Pulling images and mask

### We'll first do this with one patient

In [11]:
patient_data_path = os.path.join(data_path,'ABD_LYMPH_036')
Dicom_Reader.Make_Contour_From_directory(patient_data_path)
print('Done!')

Done!


## View images

In [12]:
%matplotlib inline

### The images and mask are saved within the Dicom_Reader class, so we just have to load them

In [13]:
Images = Dicom_Reader.ArrayDicom
mask = Dicom_Reader.mask # This is the mask

#### Threshold

In [14]:
Images[Images<-200] = -200
Images[Images>200] = 200

In [ ]:
plot_Image_Scroll_Bar_Image(Images)

interactive(children=(IntSlider(value=0, description='Z', max=160), Text(value='2D', description='view'), Outp…

In [16]:
Images[mask==1] += 300

## Recap

### Checking ROI contour names and making associations

### Loading in image and mask from desired contour name

### Viewing images and mask

## Separate into Train/Test/Validation

### This is also important, but I would recommend using the .write_parallel function
### For ease, this has already been done for you

In [ ]:
desc = 'TCIA_Liver_Patients'
Dicom_Reader.set_get_images_and_mask(False)
Dicom_Reader.set_description(desc)
Dicom_Reader.down_folder(data_path,reset=True)
Dicom_Reader.write_parallel(os.path.join(data_path,'..','Niftii_Arrays'),excel_file=os.path.join('.','patient_dist.xlsx'))

## Take that date and distribute it into a training/validation/test as you like

In [25]:
output_path = data_path.replace('Whole_Patients','Niftii_Arrays')
#write_data(data_path,output_path, Dicom_Reader)
#Separate_files(output_path) # Separate into a Training/Validation/Test set
#run_main(output_path,extension=5)

## Turn into TFRecords

In [27]:
from Pre_Processing.Make_Single_Images.Make_TFRecord_Class import write_tf_record
from Pre_Processing.Make_Single_Images.Image_Processors_Module.Image_Processors_TFRecord import *
image_processors = [To_Categorical(),
                    Cast_Data({"image":"float16",'annotation':'int8'}),
                    Distribute_into_2D()]
path = data_path.replace('Whole_Patients','Niftii_Arrays')
write_tf_record(os.path.join(path, 'Train'), out_path=os.path.join(path,'Records','Train'), image_processors=image_processors,
                is_3D=False, verbose=False)
image_processors = [To_Categorical(),
                    Cast_Data({"image":"float16",'annotation':'int8'}),
                    Distribute_into_3D()]
write_tf_record(os.path.join(path, 'Validation'), out_path=os.path.join(path,'Records','Validation'), image_processors=image_processors,
                is_3D=True)
image_processors = [To_Categorical(),
                    Cast_Data({"image":"float16",'annotation':'int8'}),
                    Distribute_into_3D()]
write_tf_record(os.path.join(path, 'Test'), out_path=os.path.join(path,'Records','Test'), image_processors=image_processors,
                is_3D=True)

.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_0.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_1.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_10.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_11.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_12.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_13.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_14.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_2.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_3.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_4.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_5.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_6.nii.gz
.\..\..\Data\Niftii_Arrays\Train\Overall_Data_TCIA_Liver_Patients_7.nii.gz
.\..\..\Data\Niftii_